In [6]:
import polars as pl
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [7]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [8]:
train_basetable = pl.read_csv("train/train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv("train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv("train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv("train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv("train/train_person_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv("train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [9]:
test_basetable = pl.read_csv("test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv("test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv("test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv("test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv("test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv("test/test_person_1.csv").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_csv("test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [10]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

# # We will process in this examples only A-type and M-type columns, so we need to select them.
# selected_static_cols = []
# for col in train_static.columns:
#     if col[-1] in ("A", "M"):
#         selected_static_cols.append(col)
# print(selected_static_cols)

# selected_static_cb_cols = []
# for col in train_static_cb.columns:
#     if col[-1] in ("A", "M"):
#         selected_static_cb_cols.append(col)
# print(selected_static_cb_cols)

# # Join all tables together.
# data = train_basetable.join(
#     train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
# ).join(
#     train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
# ).join(
#     train_person_1_feats_1, how="left", on="case_id"
# ).join(
#     train_person_1_feats_2, how="left", on="case_id"
# ).join(
#     train_credit_bureau_b_2_feats, how="left", on="case_id"
# )

selected_static_cols = [col for col in train_static.columns if col != 'case_id']
selected_static_cb_cols = [col for col in train_static_cb.columns if col != 'case_id']

data = train_basetable.join(
    train_static.select(["case_id"] + selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"] + selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [11]:
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [12]:
case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

['actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'bankacctype_710L', 'cardtype_51L', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'currdebt_22A',

In [13]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

Train: (915995, 219)
Valid: (305332, 219)
Test: (305332, 219)


In [14]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

model_gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

c:\Users\iswzh\anaconda3\envs\cs567proj\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.772107
[100]	valid_0's auc: 0.788719
[150]	valid_0's auc: 0.795784
[200]	valid_0's auc: 0.799792
[250]	valid_0's auc: 0.802754
[300]	valid_0's auc: 0.804629
[350]	valid_0's auc: 0.806144
[400]	valid_0's auc: 0.807258
[450]	valid_0's auc: 0.808129
[500]	valid_0's auc: 0.809039
[550]	valid_0's auc: 0.809982
[600]	valid_0's auc: 0.810826
[650]	valid_0's auc: 0.811336
[700]	valid_0's auc: 0.812177
[750]	valid_0's auc: 0.81253
[800]	valid_0's auc: 0.813143
[850]	valid_0's auc: 0.813426
[900]	valid_0's auc: 0.813762
[950]	valid_0's auc: 0.813971
Early stopping, best iteration is:
[985]	valid_0's auc: 0.814199


In [15]:

# 对类别特征进行独热编码
X_train_encoded = pd.get_dummies(X_train)
X_valid_encoded = pd.get_dummies(X_valid)
X_test_encoded = pd.get_dummies(X_test)


# 确保所有数据集具有相同的列
X_train_encoded, X_valid_encoded = X_train_encoded.align(X_valid_encoded, join='outer', axis=1)
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='outer', axis=1)
X_valid_encoded, X_test_encoded = X_valid_encoded.align(X_test_encoded, join='outer', axis=1)

# 用 0 填充 NaN 值
X_train_encoded = X_train_encoded.fillna(0)
X_valid_encoded = X_valid_encoded.fillna(0)
X_test_encoded = X_test_encoded.fillna(0)


# 创建 XGBoost 模型实例
model_XGBoost = XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=3)

model_XGBoost.fit(X_train_encoded, y_train)


In [ ]:

# 训练随机森林模型
model_random_forest = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=42, n_jobs=-1)
model_random_forest.fit(X_train_encoded, y_train)


RandomForestClassifier(max_depth=3, n_estimators=1000, n_jobs=-1,
                       random_state=42)

In [ ]:

# 训练岭回归模型
model_ridge = Ridge(alpha=1.0, random_state=42)

model_ridge.fit(X_train_encoded, y_train)


c:\Users\iswzh\anaconda3\envs\cs567proj\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.58132e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(random_state=42)

In [ ]:
# Create a function to generate meta-features
def generate_meta_features(model, X_encoded, X):
    y_pred_proba = model.predict_proba(X_encoded)[:, 1] if hasattr(model, 'predict_proba') else model.predict(X_encoded)
    return y_pred_proba

# Generate meta-features for the train, valid, and test sets
meta_features_train = np.column_stack([
    generate_meta_features(model_random_forest, X_train_encoded, X_train),
    generate_meta_features(model_ridge, X_train_encoded, X_train),
    generate_meta_features(model_XGBoost, X_train_encoded, X_train),
    model_gbm.predict(X_train, num_iteration=model_gbm.best_iteration)
])

meta_features_valid = np.column_stack([
    generate_meta_features(model_random_forest, X_valid_encoded, X_valid),
    generate_meta_features(model_ridge, X_valid_encoded, X_valid),
    generate_meta_features(model_XGBoost, X_valid_encoded, X_valid),
    model_gbm.predict(X_valid, num_iteration=model_gbm.best_iteration)
])

from sklearn.ensemble import GradientBoostingRegressor  # For regression

meta_model = GradientBoostingRegressor()  # or GradientBoostingClassifier() for classification

meta_model.fit(meta_features_valid, base_valid["target"])

# Generate meta-features for the test set and make final predictions
meta_features_test = np.column_stack([
    generate_meta_features(model_random_forest, X_test_encoded, X_test),
    generate_meta_features(model_ridge, X_test_encoded, X_test),
    generate_meta_features(model_XGBoost, X_test_encoded, X_test),
    model_gbm.predict(X_test, num_iteration=model_gbm.best_iteration)
])

# Generate meta-features for the train set and make final predictions
final_predictions_train = meta_model.predict(meta_features_train)
base_train["score"] = final_predictions_train

# Generate meta-features for the valid set and make final predictions
final_predictions_valid = meta_model.predict(meta_features_valid)
base_valid["score"] = final_predictions_valid

# Generate meta-features for the test set and make final predictions
final_predictions_test = meta_model.predict(meta_features_test)
base_test["score"] = final_predictions_test
# Calculate AUC scores
print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], meta_model.predict(meta_features_train))}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], meta_model.predict(meta_features_valid))}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')


The AUC score on the train set is: 0.7639849865945045
The AUC score on the valid set is: 0.7553796054166325
The AUC score on the test set is: 0.7484253338300456


In [ ]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}')  

The stability score on the train set is: 0.4971429154444473
The stability score on the valid set is: 0.48075094467694374
The stability score on the test set is: 0.4590132528157713


In [ ]:
X_submission = data_submission[cols_pred].to_pandas()
X_submission = convert_strings(X_submission)
categorical_cols = X_train.select_dtypes(include=['category']).columns

for col in categorical_cols:
    train_categories = set(X_train[col].cat.categories)
    submission_categories = set(X_submission[col].cat.categories)
    new_categories = submission_categories - train_categories
    X_submission.loc[X_submission[col].isin(new_categories), col] = "Unknown"
    new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
    X_train[col] = X_train[col].astype(new_dtype)
    X_submission[col] = X_submission[col].astype(new_dtype)

# 对类别特征进行独热编码
X_submission_encoded = pd.get_dummies(X_submission)
# 确保特征对齐
X_submission_encoded = X_submission_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
# 用 0 填充 NaN 值
X_submission_encoded = X_submission_encoded.fillna(0)

# Generate meta-features for the submission data
meta_features_submission = np.column_stack([
    model_random_forest.predict_proba(X_submission_encoded)[:, 1],
    model_ridge.predict(X_submission_encoded),
    model_XGBoost.predict_proba(X_submission_encoded)[:, 1],
    model_gbm.predict(X_submission)
])

# Use the meta-model to make final predictions on the submission data
y_submission_pred = meta_model.predict(meta_features_submission)


In [ ]:
submission = pd.DataFrame({
    "case_id": data_submission["case_id"].to_numpy(),
    "score": y_submission_pred
}).set_index('case_id')
submission.to_csv("./submission_stacking_v1.csv")

In [ ]:
# 假设已有的代码中，base_test, X_test, y_test, meta_model 已经准备好
# 进行模型的预测
y_pred = meta_model.predict(meta_features_test)

# 将预测结果添加到base_test数据框中
base_test["predicted"] = y_pred

# 找出错误的预测
errors = base_test[base_test["predicted"] != base_test["target"]]

# 分析错误预测中每个特征名称的字母后缀的出现频率
def analyze_suffixes(df):
    suffix_count = {}
    for col in df.columns:
        # 只关心原始特征，忽略其他添加的列如 'case_id', 'predicted', 'target'
        if col not in ['case_id', 'predicted', 'target', 'score']:
            suffix = col[-1]
            if suffix in suffix_count:
                suffix_count[suffix] += 1
            else:
                suffix_count[suffix] = 1
    return suffix_count

# 调用函数并打印结果
suffix_frequency = analyze_suffixes(errors)
print("Suffix frequency in misclassified samples:", suffix_frequency)


Suffix frequency in misclassified samples: {'M': 1}


In [ ]:
# 打印出所有使用的特征的后缀，以检查是否都被正确处理
all_suffixes = {col[-1] for col in X_test.columns if col not in ['case_id', 'predicted', 'target', 'score']}
print("All suffixes in the feature set:", all_suffixes)


All suffixes in the feature set: {'A', 'M'}
